# Decision Trees

Like SVMs, Decision Trees are versatile Machine Learning algorithms that can perform both classification and regression tasks.

## Training and Visualizing a Decision Tree

In [23]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
rc('text', usetex = True)
rc('font', family='serif')

import os
def image_path(fig_id):
    return os.path.join(fig_id)

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

We can visualize the tree using __export_graphviz()__ method to output a graph:

In [17]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file=image_path('iris_tree.dot'),
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

![vahid](iris_tree.png)

Node's __gini__ attribute measures its impurity. A node is __pure__ (gini = 0) if all training instances it applies to belong to the same class. 
$$G_i = 1 - \sum_{k=1}^{n}{p_{i,k}^2}$$
where $p_{i,k}$ is the ratio of class $k$ instances among the training instances in the $i$th node of the tree. For example, the depth-2 left node has a __gini__ equal to $1 - (0/54)^2 - (49/54)^2 - (5/54)^2 \approx 0.168$

### Estimating Class Probabilities

A decision tree can also estimate the probability that an instance belongs to a particular class $k$. It traverses the tree to find the leaf node for this instance, and then it returns the ratio of training instances of class $k$ in this node.

In [18]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

Here is the following probabilities: $(0/54) = 0\%, (49/54) = 90\%, (5/54) = 9.3\%$

In [19]:
tree_clf.predict([[5, 1.5]])

array([1])

## The CART Training Algorithm

Scikit-Learn uses the __Classification And Regression Tree(CART)__ algorithm to train Decision Trees. The idea is really quite simple. The algorithm first splits the training set in two subsets using a single feature $k$ and a threashold $t_k$. For each node in the tree, therefore, it requires a complexity of $O(m \times n \log(m))$.

Instead of __Gini Impurity__ we could also use __Gini entropy__ which is equal to $H_i = - \sum_{k = 1, p_{i,k \neq = 0}}^{n}{p_{i,k}\log(p_{i,k})}$. However, in practice it does not make too much difference, and __Gini Impurity__ is faster while __entropy__ tends to produce slightly more balanced tree.

### Regularization Hyperparameters

__DecisionTreeClassifier__ class has a few other parameters that similarly restrict the shape of the Decision Tree: __min_samples_split__(minimum numboer of samples a node must have before it can be split), __min_samples_leaf__ (the minimum number of samples a leaf must have), __min_weight_fraction_leaf__, __max_leaf_nodes__ (maximum number of leaf nodes), and __max_features__ (maximum number of features that are evaluated for splitting at each node).

Other algorithms work by first training the Decision Tree without restrictions, then pruning unncessary nodes. A $\chi^2$ test can be used to estimate the probability that the improvement is result of chance. This probability is called __p-value__. If __p-value__ is higher than a threshold, then the node is considered unncessary and its children are deleted.

## Regression

Decision Trees are also capable of performing regression tasks. 

In [24]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

export_graphviz(
    tree_reg,
    out_file=image_path('iris_tree_reg.dot'),
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

![](iris_tree_reg.png)

This tree looks very similar to the classification tree. The main difference is that instead of predicting a class in each node, it predicts a __value__. The cost function is as follows:
$$J(k, t_k) = \frac{m_{left}}{m}MSE_{left} + \frac{m_{right}}{m}MSE_{right}$$